In [1]:
# !pip install sqlalchemy
# !pip install seaborn
# !pip install pymysql

In [1]:
from sqlalchemy import create_engine,inspect
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

### Fetching Data

In [2]:
sql_creds = json.load(open("creds.json"))

In [3]:
# sql_creds["sql_creds"]

In [4]:
user = sql_creds["sql_creds"]["user"]
passw = sql_creds["sql_creds"]["passw"]
host = sql_creds["sql_creds"]["host"]
port = 3306
database = 'stock_fundamentals'
# Create an engine instance
engine = create_engine(f'mysql+pymysql://{user}:{passw}@{host}:{port}/{database}')

In [5]:
balancesheet_df = pd.read_sql('SELECT * FROM {}'.format("balancesheet"), con=engine)

In [6]:
cashflow_df = pd.read_sql('SELECT * FROM {}'.format("cashflow"), con=engine)

In [7]:
incomesheet_df = pd.read_sql('SELECT * FROM {}'.format("incomesheet"), con=engine)

In [8]:
balancesheet_df.index = balancesheet_df["Year_Date"]
incomesheet_df.index = incomesheet_df["Year_Date"]
cashflow_df.index = cashflow_df["Year_Date"]

In [9]:
balancesheet_df.columns

Index(['CurrentPortofLTDebt_CapitalLeases', 'UnrealizedGain_Loss_',
       'RetainedEarnings_AccumulatedDeficit_', 'TotalReceivablesNet',
       'TreasuryStock_Common', 'TotalLiabilitiesShareholders'Equity',
       'TotalLongTermDebt', 'AccountsPayable', 'OtherAssetsTotal',
       'OtherEarningAssetsTotal', 'TotalCommonSharesOutstanding',
       'RedeemablePreferredStockTotal', 'OtherEquityTotal', 'CommonStockTotal',
       'PrepaidExpenses', 'NoteReceivable_LongTerm',
       'OtherCurrentliabilitiesTotal', 'AdditionalPaid_InCapital',
       'ESOPDebtGuarantee', 'OtherCurrentAssetsTotal',
       'Property_Plant_EquipmentTotal_Gross',
       'PreferredStock_NonRedeemableNet', 'CashandShortTermInvestments',
       'Annual_Quarter', 'Property_Plant_EquipmentTotal_Net', 'TotalInventory',
       'OtherLiabilitiesTotal', 'TotalEquity', 'Cash', 'LongTermInvestments',
       'TotalDebt', 'CapitalLeaseObligations', 'GoodwillNet', 'IntangiblesNet',
       'TangibleBookValueperShareCommonEq', 'To

In [10]:
incomesheet_df.columns

Index(['NetInterestIncome', 'LossesBenefitsandAdjustmentsTotal',
       'DilutedWeightedAverageShares', 'OperatingIncome',
       'TotalAdjustmentstoNetIncome', 'InterestExp_Inc_Net_OperatingTotal',
       'NetInterestIncAfterLoanLossProv', 'Gain_Loss_onSaleofAssets',
       'NetIncomeBeforeExtraItems', 'TotalRevenue',
       'OtherOperatingExpensesTotal', 'Annual_Quarter', 'NetIncomeAfterTaxes',
       'ResearchDevelopment', 'OtherNet', 'IncomeAvailabletoComInclExtraOrd',
       'Non_InterestIncomeBank', 'RealizedUnrealizedGains_Losses_',
       'Depreciation_Amortization', 'InterestIncomeBank',
       'DilutedNormalizedEPS', 'Non_InterestExpenseBank', 'stockName',
       'GrossProfit', 'TotalOperatingExpense',
       'IncomeAvailabletoComExclExtraOrd', 'EquityInAffiliates',
       'NetInvestmentIncome', 'NetIncome', 'DilutedEPSExcludingExtraOrdItems',
       'InterestInc_Exp_Net_Non_OpTotal', 'OperationsMaintenance',
       'CostofRevenueTotal', 'DilutionAdjustment', 'FuelExpense',
 

In [11]:
piotroski_dict = {}

## Calculation

### ROA

In [12]:
def roa_calculation(i_df,b_df,c_df):
    roa_df = pd.merge(i_df["NetIncomeBeforeExtraItems"], b_df["TotalAssets"], left_index=True, right_index=True)
    roa_df.dropna(inplace=True)
    roa_df = roa_df.astype(float)
    roa_df["roa"] = roa_df["NetIncomeBeforeExtraItems"]/roa_df["TotalAssets"]
    roa_df.dropna(inplace = True)
    roa_score = [1 if i > 0 else 0 for i in list(roa_df["roa"])]
    roa_df["roa_score"] = roa_score
    return roa_df["roa_score"],roa_df

### CFO

In [13]:
def cfo_calculation(i_df,b_df,c_df):
    cfo_df = pd.merge(b_df["TotalAssets"],c_df['CashfromOperatingActivities'], left_index=True, right_index=True)
    cfo_df = cfo_df.astype(float)
    cfo_df["cfo"] = cfo_df["CashfromOperatingActivities"] / cfo_df["TotalAssets"]
    cfo_df.dropna(inplace = True)
    cfo_score = [1 if i > 0 else 0 for i in list(cfo_df["cfo"])]
    cfo_df["cfo_score"] = cfo_score
    return cfo_df["cfo_score"],cfo_df

### Delta function

In [14]:
def delta_calculation(df,name):
    yearList = []
    quarterList = []
    for date in df.index:
        if len(date) == 4:
            yearList.append(date)
        else:
            quarterList.append(date)
    delta = []
    for idx in range(0,len(yearList)-1):
        delta.append(df.loc[yearList[idx]][name] - df.loc[yearList[idx+1]][name])
    delta.append(df.loc[yearList[-1]][name] - 0)
    for idx in range(0,len(quarterList)-1):
        delta.append(df.loc[quarterList[idx]][name] - df.loc[quarterList[idx+1]][name])
    # yearList = list(map(int, yearList))
    delta.append(df.loc[quarterList[-1]][name] - 0)
    yearList.extend(quarterList)
    return yearList,delta

### Delta ROA

In [15]:
def delta_roa_calculation(roa_df):
    delta_roa_df = pd.DataFrame()
    yearList,delta_roa = delta_calculation(roa_df,"roa")
    delta_roa_df.index = yearList
    delta_roa_df["delta_roa"] = delta_roa
    
    delta_roa_score = [1 if i > 0 else 0 for i in list(delta_roa_df["delta_roa"])]
    delta_roa_df["delta_roa_score"] = delta_roa_score
    return delta_roa_df["delta_roa_score"]

### Accrual

In [16]:
def accrual_calculation(roa_df,cfo_df):
    accrual_df = pd.DataFrame()
    accrual_df["roa"] = roa_df["roa"]
    accrual_df["cfo"] = cfo_df.dropna()["cfo"]
    accrual_df["accrual"] = roa_df["roa"] - cfo_df.dropna()["cfo"]
    accrual_df.dropna(inplace = True)
    accrual_score = [0 if i > 0 else 1 for i in list(accrual_df["accrual"])]
    accrual_df["accrual_score"] = accrual_score
    return accrual_df["accrual_score"]

### Delta Leverage

In [17]:
def delta_leverage_calculation(i_df,b_df,c_df):
    lev_df = pd.merge(b_df["TotalAssets"],b_df['TotalLongTermDebt'], left_index=True, right_index=True)
    lev_df = lev_df.astype(float)
    lev_df["lev"] = lev_df["TotalLongTermDebt"] / lev_df["TotalAssets"]
    lev_df.dropna(inplace = True)
    
    delta_lev_df = pd.DataFrame()
    yearList,delta_lev = delta_calculation(lev_df,"lev")
    delta_lev_df.index = yearList
    delta_lev_df["delta_lev"] = delta_lev
    
    delta_lev_score = [0 if i > 0 else 1 for i in list(delta_lev_df["delta_lev"])]
    delta_lev_df["delta_lev_score"] = delta_lev_score
    return delta_lev_df["delta_lev_score"]

### Delta Liquid

In [18]:
def delta_liquid_calculation(i_df,b_df,c_df):
    liquid_df = pd.merge(b_df["TotalCurrentLiabilities"],b_df["TotalCurrentAssets"],left_index=True,right_index=True)
    liquid_df = liquid_df.astype(float)
    liquid_df["liquid"] = liquid_df["TotalCurrentAssets"] / liquid_df["TotalCurrentLiabilities"]
    
    liquid_df.dropna(inplace = True)
    
    delta_liquid_df = pd.DataFrame()
    if len(liquid_df) == 0:
        liquid_df = pd.merge(b_df["TotalLiabilities"],b_df["TotalAssets"],left_index=True,right_index=True)
        liquid_df = liquid_df.astype(float)
        liquid_df["liquid"] = liquid_df["TotalAssets"] / liquid_df["TotalLiabilities"]
    liquid_df.dropna(inplace = True)
    if len(liquid_df) == 0:
        return 0
    yearList,delta_liquid = delta_calculation(liquid_df,"liquid")
    delta_liquid_df.index = yearList
    delta_liquid_df["delta_liquid"] = delta_liquid
    
    delta_liquid_score = [1 if i > 0 else 0 for i in list(delta_liquid_df["delta_liquid"])]
    delta_liquid_df["delta_liquid_score"] = delta_liquid_score
    return delta_liquid_df["delta_liquid_score"]

### EQ offer

In [19]:
def eq_offer_calculation(i_df,b_df,c_df):
    eq_df = pd.DataFrame()
    eq_df["eq"] = b_df["TotalCommonSharesOutstanding"]
    eq_df = eq_df.astype(float)
    
    
    eq_df.dropna(inplace = True)
    
    delta_eq_df = pd.DataFrame()
    yearList,delta_eq = delta_calculation(eq_df,"eq")
    delta_eq_df.index = yearList
    delta_eq_df["delta_eq"] = delta_eq
    
    delta_eq_score = [0 if i > 0 else 1 for i in list(delta_eq_df["delta_eq"])]
    delta_eq_df["delta_eq_score"] = delta_eq_score
    return delta_eq_df["delta_eq_score"]

### Delta Margin

In [20]:
def delta_margin_calculation(i_df,b_df,c_df):
    margin_df = pd.merge(i_df["GrossProfit"],i_df["TotalRevenue"],left_index=True,right_index=True)
    margin_df = margin_df.astype(float)
    margin_df["margin"] = (margin_df["GrossProfit"]/margin_df["TotalRevenue"]) * 100
    margin_df.dropna(inplace = True)
    
    delta_margin_df = pd.DataFrame()
    if len(margin_df) == 0:
        margin_df = pd.merge(i_df["InterestIncomeBank"],i_df["Non_InterestIncomeBank"],left_index=True,right_index=True)
        margin_df = pd.merge(margin_df,i_df["NetInterestIncome"],left_index=True,right_index=True)
        margin_df = margin_df.astype(float)
        margin_df["margin"] = margin_df["NetInterestIncome"] / (margin_df["Non_InterestIncomeBank"] + margin_df["InterestIncomeBank"])
    margin_df.dropna(inplace = True)
    if len(margin_df) == 0:
        return 0
    yearList,delta_margin = delta_calculation(margin_df,"margin")
    delta_margin_df.index = yearList
    delta_margin_df["delta_margin"] = delta_margin
    
    delta_margin_score = [1 if i > 0 else 0 for i in list(delta_margin_df["delta_margin"])]
    delta_margin_df["delta_margin_score"] = delta_margin_score
    return delta_margin_df["delta_margin_score"]

### Delta Turnover

In [21]:
def delta_turnover_calculation(i_df,b_df,c_df):
    turnover_df = pd.merge(b_df["TotalAssets"],i_df["TotalRevenue"],left_index=True,right_index=True)
    turnover_df = turnover_df.astype(float)
    turnover_df["turnover"] = turnover_df["TotalRevenue"] / turnover_df["TotalAssets"]
    
    turnover_df.dropna(inplace = True)
    
    delta_turnover_df = pd.DataFrame()
    if len(turnover_df) == 0:
        turnover_df = pd.merge(b_df["TotalAssets"],i_df["Non_InterestIncomeBank"],left_index=True,right_index=True)
        turnover_df = pd.merge(turnover_df,i_df["InterestIncomeBank"],left_index=True,right_index=True)
        turnover_df = turnover_df.astype(float)
        turnover_df["turnover"] = (turnover_df["Non_InterestIncomeBank"] + turnover_df["InterestIncomeBank"]) / turnover_df["TotalAssets"]
    turnover_df.dropna(inplace = True)
    if len(turnover_df) == 0:
        return 0
    yearList,delta_turnover = delta_calculation(turnover_df,"turnover")
    delta_turnover_df.index = yearList
    delta_turnover_df["delta_turnover"] = delta_turnover
    
    delta_turnover_score = [1 if i > 0 else 0 for i in list(delta_turnover_df["delta_turnover"])]
    delta_turnover_df["delta_turnover_score"] = delta_turnover_score
    return delta_turnover_df["delta_turnover_score"]

## Store Data

In [22]:
merge_error = []
calc_error = []
stockName = list(incomesheet_df["stockName"].unique())
final_df = pd.DataFrame()
for idx,stock in enumerate(stockName):
    print(f'\rProgress: {idx}/{len(stockName)}', end='', flush=True)
    df = pd.DataFrame()
    i_df = incomesheet_df[incomesheet_df["stockName"] == stock]
    b_df = balancesheet_df[balancesheet_df["stockName"] == stock]
    c_df = cashflow_df[cashflow_df["stockName"] == stock]
    try:
        roa,roa_df = roa_calculation(i_df,b_df,c_df)
        cfo,cfo_df = cfo_calculation(i_df,b_df,c_df)
        delta_roa = delta_roa_calculation(roa_df)
        accrual = accrual_calculation(roa_df,cfo_df)
        delta_leverage = delta_leverage_calculation(i_df,b_df,c_df)
        delta_liquid = delta_liquid_calculation(i_df,b_df,c_df)
        eq_offer = eq_offer_calculation(i_df,b_df,c_df)
        delta_margin = delta_margin_calculation(i_df,b_df,c_df)
        delta_turnover = delta_turnover_calculation(i_df,b_df,c_df)
        # print(roa,cfo,delta_roa,accrual,delta_leverage,delta_liquid,eq_offer,delta_margin,delta_turnover)
    except Exception as e:
        # print("error calculation : {}".format(stock))
        calc_error.append(stock)
        continue
    metricsList = [roa,cfo,delta_roa,accrual,delta_leverage,delta_liquid,eq_offer,delta_margin,delta_turnover]
    try:
        df = metricsList[0]
        for i in range(1,len(metricsList)):
            df = pd.merge(metricsList[i],df,left_index=True,right_index=True)
    except Exception as e:
        # print("error merge : {}".format(stock))
        merge_error.append(stock)
        continue
    df["final_score"] = df.sum(axis = 1)
    df["stock"] = stock
    df["date/year"] = df.index
    final_df = pd.concat([final_df,df],ignore_index=True,axis = 0)


Progress: 591/1779

C:\Users\Akshay\AppData\Local\Temp\ipykernel_22900\2678638393.py:11: RuntimeWarning: invalid value encountered in scalar subtract
  delta.append(df.loc[yearList[idx]][name] - df.loc[yearList[idx+1]][name])


Progress: 1034/1779

C:\Users\Akshay\AppData\Local\Temp\ipykernel_22900\2678638393.py:11: RuntimeWarning: invalid value encountered in scalar subtract
  delta.append(df.loc[yearList[idx]][name] - df.loc[yearList[idx+1]][name])


Progress: 1209/1779

...

Progress: 1580/1779

C:\Users\Akshay\AppData\Local\Temp\ipykernel_22900\2678638393.py:14: RuntimeWarning: invalid value encountered in scalar subtract
  delta.append(df.loc[quarterList[idx]][name] - df.loc[quarterList[idx+1]][name])
C:\Users\Akshay\AppData\Local\Temp\ipykernel_22900\2678638393.py:14: RuntimeWarning: invalid value encountered in scalar subtract
  delta.append(df.loc[quarterList[idx]][name] - df.loc[quarterList[idx+1]][name])


Progress: 1704/1779

C:\Users\Akshay\AppData\Local\Temp\ipykernel_22900\2678638393.py:14: RuntimeWarning: invalid value encountered in scalar subtract
  delta.append(df.loc[quarterList[idx]][name] - df.loc[quarterList[idx+1]][name])


Progress: 1778/1779

In [23]:
final_df.to_sql("piotroskiscore", con=engine, if_exists='replace', index=False)

15314

In [24]:
final_df["id"] = final_df.index

In [25]:
from datetime import date
 
# Returns the current local date
today = date.today()
final_df.to_csv("./Piotroski Score Reports/piotroski_score_{}.csv".format(today))